In [28]:
import requests
import pandas as pd
import yfinance as yf

polygon_api_key = 'JOLy_7GmGcMczPwKGCu5EtrZ40CAkeqt'

def get_stock_splits(start_date, end_date):
    base_url = 'https://api.polygon.io/v3/reference/splits'
    params = {
        'apiKey': polygon_api_key,
        'start_date': start_date,
        'end_date': end_date,
        'limit': 1000  # Adjust the limit as needed
    }
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        return data.get('results', [])
    except requests.exceptions.RequestException as e:
        print(f"HTTP Request failed: {e}")
        return []

# Read the existing CSV file
file_path = 'Robinhood_Profit.csv'  # Update this with the actual path to your CSV file
df_existing = pd.read_csv(file_path)

# Collect symbols
symbols = df_existing['Symbol'].unique()

# Example usage
start_date = '2000-01-01'  # Expanded date range to ensure capturing all relevant splits
end_date = '2024-12-31'
splits = get_stock_splits(start_date, end_date)

# Convert the result to a DataFrame for easier handling
df_splits = pd.DataFrame(splits)

# Ensure the 'execution_date' column is in datetime format
df_splits['execution_date'] = pd.to_datetime(df_splits['execution_date'])

# Filter the DataFrame to include only rows with the symbols from the existing CSV
df_splits = df_splits[df_splits['ticker'].isin(symbols)]

# Remove rows where split_from is greater than or equal to split_to
df_splits = df_splits[df_splits['split_from'] < df_splits['split_to']]

# Format the DataFrame as per the new requirement
df_splits['Date'] = df_splits['execution_date'].dt.strftime('%Y-%m-%d')
df_splits['Symbol'] = df_splits['ticker']
df_splits['Split Ratio'] = df_splits['split_from'].astype(int).astype(str) + ':' + df_splits['split_to'].astype(int).astype(str)

# Select and reorder the columns as required
df_splits = df_splits[['Date', 'Symbol', 'Split Ratio']]

# Save to a new CSV file
output_path = 'new_splittin_data.csv'  # Update this with the desired path to your new CSV file
df_splits.to_csv(output_path, index=False)

# Display the dataframe
df_splits


,Date,Symbol,Split Ratio
481,2024-03-04,LQR,1:1
562,2024-02-05,MMAT,1:2
